# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [66]:
# imports
import glob
import os 
import math

import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import f1_score
from IPython.display import display


In [67]:
!git clone https://github.com/kfirbar/nlp-course.git

fatal: destination path 'nlp-course' already exists and is not an empty directory.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [68]:

!ls nlp-course/lm-languages-data-new


en.csv     es.json    in.csv     it.json    pt.csv     test.json  tl.csv
en.json    fr.csv     in.json    nl.csv     pt.json    tests.csv  tl.json
es.csv     fr.json    it.csv     nl.json    test.csv   tests.json


In [69]:
data_files = {'en_df': 'en.csv',
              'es_df': 'es.csv',
              'fr_df': 'fr.csv',
              'in_df': 'in.csv',
              'it_df': 'it.csv',
              'nl_df': 'nl.csv',
              'pt_df': 'pt.csv',
              'tl_df': 'tl.csv'}

    
directory = 'nlp-course/lm-languages-data-new/'    
for (key, value) in data_files.items():
    data_files[key] = directory + value
    
languages_list = list(data_files.keys())
start_token = '↠'
end_token = '↞'

**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [70]:
def preprocess():
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """  
    tokens = []
    for path in data_files.values():
        df = pd.read_csv(path)
        for text in df['tweet_text'].values:
            tokens.extend(list(text))
    return list(set(tokens))

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [71]:
#helper functions
def tweets_to_text(data_file_path, n):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """
    df = pd.read_csv(r''+ data_file_path)
    tweets_list = df['tweet_text'].apply(lambda x: start_token + x + end_token).values
    text = ''.join(tweets_list)
    
    text = start_token * (n-1) + text + end_token * (n-1)

    return text

def reorder_list(List, index_list):
    return [List[i] for i in index_list]

In [72]:
def lm(n, vocabulary, data_file_path, add_one):
    # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
    # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
    # data_file_path - the data_file from which we record probabilities for our model
    # add_one - True/False (use add_one smoothing or not)
  
    lm_dict = {}
    V = len(vocabulary)

    text = tweets_to_text(data_file_path, n)

    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]

    lm_dict = defaultdict(lambda: defaultdict(lambda: 0))

    for i_n_gram in n_gram:
        n_1_gram = i_n_gram[0:n-1]
        lm_dict[n_1_gram][i_n_gram[n-1]] += 1
    
    for key in lm_dict.keys():
        key_count = sum(lm_dict[key].values())
        inner_dict = {}
        for key_1 in lm_dict[key].keys():
            if add_one:
                inner_dict[key_1] = (lm_dict[key][key_1] + 1) / (key_count + V)
            else:
                inner_dict[key_1] = lm_dict[key][key_1]/ key_count
        if add_one:
            lm_dict[key] = defaultdict(lambda: 1 / (key_count + V), inner_dict)
        else:
            lm_dict[key] = defaultdict(lambda: 0, inner_dict)
            
    return lm_dict

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [73]:
def eval_tweet(n, N, model, tweet):
  missing_value = 1e-8
  tweet_probabilities = []
  
  for i in range(N - n):
    i_n_gram = tweet[i: i + n]
    key = i_n_gram[0:n-1]
    key_1 = i_n_gram[n-1]

    if key in model:
      if key_1 in model[key]:
        tweet_probabilities.append(model[key][key_1])
      else:
        tweet_probabilities.append(missing_value)
    else:
      tweet_probabilities.append(missing_value)
  
  return tweet_probabilities

In [74]:
def eval(n, model, data_file):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to calculate a perplexity score for
  
  df = pd.read_csv(data_file)
  probabilities = []

  for tweet in df['tweet_text'].values:
    tweet = start_token + tweet + end_token
    N = len(tweet)
    tweet_probabilities = eval_tweet(n, N, model, tweet)
    probabilities.extend(tweet_probabilities)
    
  entropy = -math.log2(np.mean(probabilities))
      
  return 2 ** entropy

In [75]:
vocabulary = preprocess()
n = 2
lm_nodel = lm(n, vocabulary, data_files['en_df'], False)
eval(n,lm_nodel, data_files['en_df'])


8.548435563954964

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [76]:
def match(n, add_one, data_files):
    # n - the n-gram to use for creating n-gram models
    # add_one - use add_one smoothing or not
    result_dict = {}
    vocabulary = preprocess()
    for i_language_model in languages_list:
        
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)
        result_dict[i_language_model] = {}

        for i_language_test in languages_list:
            i_language_model_i_score = eval(n, i_model, data_files[i_language_test])
            result_dict[i_language_model][i_language_test] = i_language_model_i_score
    perlexity_df = pd.DataFrame(result_dict)
    return perlexity_df  

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [77]:
 
def run_match(data_files):

    for n in range(1,5):
        add_one = True
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)

        add_one = False
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)



# run the model generation

In [78]:
model_dict = run_match(data_files)


n = 1, add_one = True


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,22.123184,21.734873,21.950330,23.377966,22.626058,22.589962,21.898583,24.073582
es_df,21.719854,20.354689,20.987767,22.129104,21.467562,21.883604,20.537125,22.820298
fr_df,21.959014,21.010591,21.020654,23.121476,22.109180,21.977154,21.269963,24.125203
in_df,23.336182,22.104818,23.071022,21.130734,22.944058,23.682041,22.111354,22.145412
it_df,22.598198,21.455954,22.073222,22.956840,21.951552,22.842744,21.611497,23.615910
nl_df,22.582835,21.891828,21.961533,23.716959,22.863691,21.925337,22.260563,24.737133
pt_df,21.812543,20.470572,21.178008,22.063923,21.553124,22.180099,20.410400,22.749803
tl_df,24.000402,22.766652,24.042356,22.117618,23.573142,24.669722,22.770099,22.617781


n = 1, add_one = False


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,22.077733,21.687776,21.906666,23.323182,22.575065,22.542423,21.839575,24.012277
es_df,21.675219,20.310549,20.945995,22.077205,21.419149,21.837532,20.481743,22.762143
fr_df,21.913893,20.965044,20.978818,23.067283,22.059336,21.930889,21.212628,24.063765
in_df,23.288270,22.056929,23.025158,21.081154,22.892357,23.632234,22.051779,22.088956
it_df,22.551781,21.409453,22.029316,22.903029,21.902061,22.794679,21.553251,23.555753
nl_df,22.536451,21.844394,21.917847,23.661391,22.812169,21.879180,22.200591,24.674162
pt_df,21.767717,20.426179,21.135860,22.012173,21.504516,22.133409,20.355356,22.691825
tl_df,23.951145,22.717347,23.994586,22.065751,23.520042,24.617867,22.708772,22.560137


n = 2, add_one = True


...

,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,7.379188,10.756152,9.696012,11.268243,10.921722,9.887505,11.786764,11.097943
es_df,10.762831,7.212687,8.926835,11.868470,9.746611,10.968389,9.273630,12.366126
fr_df,9.854113,9.263584,7.005886,11.943455,10.317397,10.176139,10.738881,12.436426
in_df,11.669429,12.131936,12.092548,8.777188,12.463657,11.609217,13.475092,11.022058
it_df,11.156424,9.960425,10.317858,12.262880,7.994792,12.225118,11.298069,12.562298
nl_df,9.333703,10.140853,9.313059,10.700899,11.135169,7.131643,11.608194,11.513278
pt_df,11.168523,8.752595,9.874979,12.340612,10.238764,11.810045,7.947916,12.838324
tl_df,11.274558,12.642175,12.613149,10.888561,12.388844,12.102080,13.789065,8.274901


n = 3, add_one = False


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,3.837019,5.448462,5.174504,5.447214,5.357470,5.129650,5.399947,5.033733
es_df,5.655030,3.838306,5.107682,5.896175,5.083818,5.665561,4.762284,5.585688
fr_df,5.329020,5.120737,3.721062,5.936231,5.306061,5.245555,5.394554,5.759871
in_df,6.261966,6.203370,6.287293,4.314009,6.251016,6.140492,6.315875,5.500943
it_df,5.833125,5.194200,5.505782,5.915753,3.900101,6.141699,5.399402,5.663712
nl_df,5.313499,5.565887,5.274628,5.606316,5.812449,3.929465,5.767885,5.626244
pt_df,5.762523,4.767150,5.468993,5.996876,5.121950,5.935856,3.632357,5.660587
tl_df,5.802303,6.123357,6.285088,5.490860,5.630289,5.876119,5.932141,3.689011


n = 4, add_one = True


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,10.008371,15.639750,14.236874,16.217838,15.801312,14.059858,17.227468,16.193827
es_df,16.703953,10.386958,13.730549,18.658325,15.278917,16.069090,14.568667,19.415137
fr_df,14.971101,13.981579,9.744129,17.856657,15.766866,14.846908,16.471562,18.710176
in_df,17.256343,18.266964,17.742980,13.182312,18.522453,17.235790,20.229874,17.771010
it_df,17.103267,15.638883,16.112398,19.098323,11.105850,17.653397,18.086056,19.628689
nl_df,12.961353,14.030025,13.072772,14.959751,14.831056,9.436660,15.961411,15.895732
pt_df,17.231312,13.473725,15.309776,19.254112,16.256408,17.468112,11.953943,19.736619
tl_df,17.361038,19.342445,18.867909,17.547877,18.718018,18.026929,20.806928,11.748046


**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [ ]:
! ls nlp-course/lm-languages-data-new

In [ ]:
test_folder = 'nlp-course/lm-languages-data-new/'
test_csv_files =  glob.glob(test_folder + '/*.csv')
test_files =  {}
for i_file in test_csv_files:
    file_name_with_ending = os.path.basename(i_file)
    file_name = os.path.splitext(file_name_with_ending)[0]
    test_files[file_name + '_df'] = f'' + i_file

In [ ]:
def match_test(n, data_file_path, add_one):
    # n - the n-gram to use for creating n-gram models
    # add_one - use add_one smoothing or not

    senstences_list = pd.read_csv(data_file_path)['tweet_text'].to_list()

    result_dict = {}

    for i_language_model in languages_list:
        result_dict[i_language_model] = {}
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)

        for i_test_senstence_idx in range(len(senstences_list)):
            i_test_senstence = senstences_list[i_test_senstence_idx]

            sentence_probabilities = eval_tweet(n, len(i_test_senstence), i_model, i_test_senstence)
            entropy = -math.log2(np.mean(sentence_probabilities))
            i_sentence_model_i_score = 2 ** entropy

            result_dict[i_language_model][i_test_senstence_idx] = i_sentence_model_i_score

    perlexity_df = pd.DataFrame(result_dict)
    display(perlexity_df)
    perlexity_array = perlexity_df.to_numpy()
    language_match_index = np.argmin(perlexity_array, axis=1)
    language_match_list = reorder_list(languages_list, language_match_index)
    perlexity_df['predict'] = language_match_index
    perlexity_df['predict_language'] = language_match_list
    display(perlexity_df)

    return perlexity_df

In [ ]:
def classify(n, data_file_path, add_one):
    match_dict  = match_test(n, data_file_path, add_one)
    return match_dict

In [ ]:
n = 2
test_path = test_folder + 'test.csv'
clasification_result = classify(2, test_path, False)

In [ ]:
y_true = pd.read_csv(test_path).get('label').to_list()
y_true = list(map(lambda x: languages_list.index(x+'_df'),y_true))
y_pred = clasification_result['predict'].to_list()

**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [ ]:
def calc_f1(y_true,y_pred ):
    return np.round(f1_score(y_true, y_pred,average="micro"),3)
f_score_result = calc_f1(y_true,y_pred)
print('The F-score we acheive is ' + str(f_score_result)+'\n')

# **Good luck!**